In [ ]:
import json
%run dataloader.ipynb
# %run dataloader_ori.ipynb

In [2]:
import random
import os
import torch.nn.functional as F
import torch
import time
import datetime
import  torchvision
import numpy as np
import pandas as pd
from PIL import Image
from torch import optim, nn
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import MultiStepLR


In [ ]:
# num_classes =9 
val_nums = np.zeros(num_classes, dtype=int)
for item in clean_val_dataset_label:
    val_nums[item] += 1
# print("val categroy mean", np.mean(val_nums, dtype=int), "category", val_nums, "precent", val_nums / np.mean(val_nums))
train_nums = np.zeros(num_classes, dtype=int)
for item in train_dataset_label:
    train_nums[item] += 1
# print("train categroy mean", np.mean(train_nums, dtype=int), "category", train_nums, "precent", np.mean(train_nums) / train_nums)

In [ ]:
class_weights = torch.FloatTensor(np.mean(train_nums) / train_nums * val_nums / np.mean(val_nums)).to(device)
model = torchvision.models.resnet50(pretrained = True)
model.fc = nn.Linear(2048,num_classes)
model.to(device)

In [6]:
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9, weight_decay=0.001)
scheduler = MultiStepLR(optimizer, milestones=[40], gamma=0.1)
ceriation = nn.CrossEntropyLoss(weight=class_weights).to(device)

In [7]:
class AverageMeter(object):
    """Computes and stores the average and current value"""

    def __init__(self, name, fmt=':f'):
        self.name = name
        self.fmt = fmt
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

    def __str__(self):
        fmtstr = '{name} {avg' + self.fmt + '}'
        return fmtstr.format(**self.__dict__)

    
class ProgressMeter(object):
    def __init__(self, num_batches, meters, prefix=""):
        self.batch_fmtstr = self._get_batch_fmtstr(num_batches)
        self.meters = meters
        self.prefix = prefix

    def display(self, batch):
        entries = [self.prefix + self.batch_fmtstr.format(batch)]
        entries += [str(meter) for meter in self.meters]
        print('\t'.join(entries))

    def _get_batch_fmtstr(self, num_batches):
        num_digits = len(str(num_batches // 1))
        fmt = '{:' + str(num_digits) + 'd}'
        return '[' + fmt + '/' + fmt.format(num_batches) + ']'
    
def accuracy(output, target, topk=(1,)):
    with torch.no_grad():
        maxk = max(topk)
        batch_size = target.size(0)

        _, pred = output.topk(maxk, 1, True, True)
        pred = pred.t()
        correct = pred.eq(target.view(1, -1).expand_as(pred))

        res = []
        for k in topk:
            correct_k = correct[:k].reshape(-1).float().sum(0, keepdim=True)
            res.append(correct_k.mul_(100.0 / batch_size))
        return res
def getTime():
    time_stamp = datetime.datetime.now()
    return time_stamp.strftime('%H:%M:%S')

In [8]:
def train(model, train_iter, ceriation, train_optimizer, num_prints=1):
    batch_time = AverageMeter('Time', ':6.3f')
    data_time = AverageMeter('Data', ':6.3f')
    losses = AverageMeter('Loss', ':6.3f')
    top1 = AverageMeter('Acc@1', ':6.2f')
    progress = ProgressMeter(
        len(train_iter),
        [batch_time, data_time, losses, top1], prefix="Train ")

    end = time.time()
    num_iter = int((len(train_iter)-1) / num_prints)
    for batch_idx in range(num_iter):
        try:
            images, labels,_ = train_iter.next()
            images = images.to(device, non_blocking=True)
            labels = labels.to(device, non_blocking=True)
        except StopIteration:
            break

        model.train()
        data_time.update(time.time() - end)
        train_optimizer.zero_grad()
        logits = model(images)
        loss = ceriation(logits, labels)
        loss.requires_grad_(True)
        loss.backward()
        train_optimizer.step()

        acc1, acc5 = accuracy(logits, labels, topk=(1, 5))
        losses.update(loss.item(), images[0].size(0))
        top1.update(acc1[0], images[0].size(0))
        batch_time.update(time.time() - end)
        end = time.time()

    progress.display(batch_idx)
    return top1.avg, losses.avg


def evaluate(model, eva_loader, ceriation, prefix, ignore=-1):
    losses = AverageMeter('Loss', ':3.2f')
    top1 = AverageMeter('Acc@1', ':3.2f')
    model.eval()

    with torch.no_grad():
        for i, (images, labels,_) in enumerate(eva_loader):
            images = Variable(images).cuda()
            labels = Variable(labels).cuda()

            logist = model(images)

            loss = ceriation(logist, labels)
            acc1, acc5 = accuracy(logist, labels, topk=(1, 5))

            losses.update(loss.item(), images[0].size(0))
            top1.update(acc1[0], images[0].size(0))

    if prefix != "":
        print(getTime(), prefix, round(top1.avg.item(), 2))

    return losses.avg, top1.avg.to("cpu", torch.float).item()

In [ ]:
best = 0
epoch_a = 50
for i in range(epoch_a):
    train_iter = iter(train_loader)
    print("Start training epoch:{}".format(i))
    train(model, train_iter, ceriation, optimizer,1)
    val_loss, val_acc = evaluate(model, val_loader, ceriation, "Epoch " + str(i) + ", Val Acc:")
    _, test_acc = evaluate(model, test_loader, ceriation, "Epoch_test "+ ", Test Acc1:")
    if test_acc>best:
        best = test_acc
        torch.save(model,'save_init_air_model')
    print(getTime(), "Init Best Test Acc1:", best)
    scheduler.step()
# The warm-up has been completed.

In [10]:
class Add2(Dataset):
    def __init__(self, Dataset,Labels,transform = None,target_transform = None):
        self.List = Dataset
        self.Labels = Labels
        self.length = len(self.List)
        if transform is None:
            self.transform = transforms.ToTensor()
        else:
            self.transform = transform
        print("NewDataset length:", self.length)
        self.target_transform = target_transform
    def __getitem__(self, index):
        img = Image.open(self.List[index]).convert("RGB")
        label = self.Labels[index]
        if self.transform is not None:
            img1 = self.transform(img)
            img2 = self.transform(img)
        if self.target_transform is not None:
            label = self.target_transform(label)
        return img1,img2,label
    def __len__(self):
        return self.length

In [ ]:
predict_dataset = Add2(train_dataset_path, train_dataset_label , transform=transform_train)
predict_loader = DataLoader(dataset=predict_dataset, batch_size=16, shuffle=False, num_workers=8, pin_memory=True, drop_last=False)

In [12]:
def predict_softmax(predict_loader, model):
    model.eval()
    softmax_outs = []
    with torch.no_grad():
        for images1, images2,_ in predict_loader:
            if torch.cuda.is_available():
                images1 = Variable(images1).cuda()
                images2 = Variable(images2).cuda()
                logits1 = model(images1)
                logits2 = model(images2)
                outputs = (F.softmax(logits1, dim=1) + F.softmax(logits2, dim=1)) / 2
                softmax_outs.append(outputs)

    return torch.cat(softmax_outs, dim=0).cpu()

In [20]:
def splite_confident(outs,noisy_targets,noisy_imagepath , Flag):
    probs, preds = torch.max(outs.data, 1)  # return the value and the index
    pred1 = preds.numpy() 
    probs1 = probs.numpy()
    confident_indexs = []
    unconfident_indexs = []
    for i in range(0, len(noisy_targets)):
        if preds[i] == noisy_targets[i] and probs1[i]>=0.5: 
            confident_indexs.append(i)
        else:
            unconfident_indexs.append(i)

    print(getTime(), "total and confident and unconfident num:",len(noisy_imagepath),len(confident_indexs), len(unconfident_indexs))
    new_confident1 = []
    new_unconfident1 = []
    new_confident_label1 = []
    new_unconfident_label1 = []
    for t in range(0,len(noisy_imagepath)):
        if t in confident_indexs:
            new_confident1.append(noisy_imagepath[t])
            new_confident_label1.append(pred1[t])
        else:
            new_unconfident1.append(noisy_imagepath[t])
            new_unconfident_label1.append(pred1[t])
    new_train_image1 = Add(new_confident1,new_confident_label1,transform=transform_train)
    new_trainloader1 = DataLoader(dataset=new_train_image1, batch_size=8, num_workers=8, pin_memory=True, shuffle=True, drop_last=True)
    return new_trainloader1,new_confident1, new_confident_label1, new_unconfident1, new_unconfident_label1, 

In [ ]:
soft_outs = predict_softmax(predict_loader, model)
new_trainloader,new_confident, new_confident_label, new_unconfident, new_unconfident_label = splite_confident(soft_outs,train_dataset_label,train_dataset_path)
clip = len(new_unconfident)/len(train_dataset_label)

In [42]:
def train_ori(model,train_iter,criterion,optimizer,clip):
    clip = 1-clip
    batch_time = AverageMeter('Time', ':6.3f')
    data_time = AverageMeter('Data', ':6.3f')
    losses = AverageMeter('Loss', ':6.3f')
    top1 = AverageMeter('Acc@1', ':6.2f')
    progress = ProgressMeter(
        len(train_iter),
        [batch_time, data_time, losses, top1], prefix="Train ")
              
    end = time.time()
    num_iter = int((len(train_iter) - 1))
    for batch_idx in range(num_iter):
        try:
            images, labels,_ = train_iter.next()
            data = images.to(device, non_blocking=True)
            labels = labels.to(device, non_blocking=True)
        except StopIteration:
            break
        model.train()
        data_time.update(time.time() - end)
        pred = model(data)
        loss = criterion(pred, labels)
        loss.requires_grad_(True)
        loss.backward()
        
# update crucial parameters        
        to_concat_g = []
        to_concat_v = []
        for name, param in model.named_parameters():
            if param.dim() in [2, 4]:
                to_concat_g.append(param.grad.data.view(-1))
                to_concat_v.append(param.data.view(-1))
        #             print(to_concat_g)
        #             print(to_concat_v) 
        all_g = torch.cat(to_concat_g)
        all_v = torch.cat(to_concat_v)
        metric = torch.abs(all_g * all_v)
        num_params = all_v.size(0)
#             print(num_params)
        nonzero_ratio = clip
        nz = int(nonzero_ratio * num_params)
        top_values, _ = torch.topk(metric, nz)
#             print(top_values)
        thresh = top_values[-1]
        for name, param in model.named_parameters():
            if param.dim() in [2, 4]:
                mask1 = (torch.abs(param.data * param.grad.data) >= thresh).type(torch.cuda.FloatTensor)
                mask2 = (torch.abs(param.data * param.grad.data) < thresh).type(torch.cuda.FloatTensor)
                mask1 = mask1 * clip
                mask2 = mask2 * (1-clip)
                mask = mask1+mask2
                param.grad.data = mask * param.grad.data
        optimizer.step()
        optimizer.zero_grad()
        acc1, acc5 = accuracy(pred, labels, topk=(1, 5))
        losses.update(loss.item(), data[0].size(0))
        top1.update(acc1[0], data[0].size(0))
        batch_time.update(time.time() - end)
        end = time.time()
    progress.display(batch_idx)
    return top1.avg, losses.avg

In [ ]:
best_acc = 0
epoch_b = 150
cll = []
for j in range(epoch_b):
    if j%20==0:
        soft_outs = predict_softmax(predict_loader, model)
        new_trainloader,new_confident, new_confident_label, new_unconfident, new_unconfident_label = splite_confident(soft_outs,train_dataset_label,train_dataset_path)
        new_clip  = clip*len(new_unconfident)/len(train_dataset_label)
        if new_clip<clip:
            clip = new_clip
    train_iter = iter(new_trainloader)
    print("Start training epoch:{}".format(j))
    train_ori(model,train_iter,ceriation,optimizer,clip)
    val_loss, val_acc = evaluate(model, val_loader, ceriation, "Epoch " + str(j) + ", Val Acc:")
    scheduler.step()
    _, test_acc = evaluate(model, test_loader, ceriation, "Epoch_test "+ ", Test Acc1:")
    if test_acc>=best_acc:
        best_acc = test_acc
        torch.save(model,'our_air_model')
    print(getTime(), "Model Best Test Acc1:", best_acc)